In [74]:
%load_ext autoreload
%autoreload 2

import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
from utils.seed import fix_seed
import random
import os
from answer_judge.vqaEval import VQAEval

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [75]:
fix_seed(0)
data_reader=ReadData('/root/autodl-tmp/hallscope/AOKVQA/train')
data=data_reader.read_all()
random.shuffle(data)

In [76]:
df=pd.DataFrame(data)

In [77]:
df.head()

,question,answer,most_likely,responses
0,[How many people is the food on the tray meant...,"[['two', 'two', 'two', 'two', 'two', 'two', 't...",{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
1,[Where is the boy most likely to be headed?\nA...,"[['out', 'sitting room', 'to kitchen', 'to wor...",{'embedding': [[[-0.01141357 0.00665283 0.00...,None
2,[What is the usual method to pay for parking h...,"[['credit card', 'card', 'card', 'credit card'...",{'embedding': [[[ 0.01519775 0.00805664 -0.01...,None
3,[How many people wearing tan pants and black s...,"[['one', 'one', 'five', 'one', 'four', 'five',...",{'embedding': [[[ 0.00099182 0.0020752 -0.00...,None
4,[Why can't people go down this road at this ti...,"[['barrier', ""it's blocked"", 'accident', 'bus ...",{'embedding': [[[-0.0402832 0.02270508 0.01...,None


In [78]:
df.shape

(3129, 4)

In [79]:
i=7

In [80]:
df.iloc[i]['most_likely']['response']

['Rock']

In [81]:
df.iloc[i]['question']

['What is the white object in the ground in front of the animals?\nAnswer the question using a single word or phrase.\n']

In [82]:
df.iloc[i]['answer']

["['gravel', 'rocks', 'rocks', 'rock', 'boulder', 'stone', 'concrete', 'rock', 'stone', 'stone']"]

In [83]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [84]:
embeddings=np.concatenate(embeddings,axis=0)

In [85]:
embeddings.shape

(3129, 29, 3584)

In [86]:
import ast

In [87]:
def transpose_lines(x):
    x=ast.literal_eval(x[0])
    return x

df['answer']=df['answer'].apply(transpose_lines)

In [88]:
df['answer']

0       [two, two, two, two, two, two, two, two, one, ...
1       [out, sitting room, to kitchen, to work, other...
2       [credit card, card, card, credit card, credit ...
3       [one, one, five, one, four, five, one, one, on...
4       [barrier, it's blocked, accident, bus accident...
                              ...                        
3124    [information, telephone, information, informat...
3125    [protection, riding, protection, plastic, plas...
3126    [on right, lady, middle woman, pale woman, blo...
3127    [nirates, mirates, dubai, emirates, dubai, sau...
3128    [fire alarm, smoke detector, smoke detector, s...
Name: answer, Length: 3129, dtype: object

In [89]:
gt_label=df['answer'].to_list()

In [90]:
gt_label[0:2]

[['two', 'two', 'two', 'two', 'two', 'two', 'two', 'two', 'one', 'two'],
 ['out',
  'sitting room',
  'to kitchen',
  'to work',
  'other room',
  'living room',
  'prom',
  'living room',
  'work',
  'work']]

In [91]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
rsps=[]
for x in responses:
    rsps+=x

In [92]:
rsps[0:5]

['1', 'bathroom', 'Credit Card', '4', 'accident']

In [93]:
judge=VQAEval(gt_label,rsps)
judge.evaluate()
print(judge.evalQA)
print(judge.accuracy['overall'])

computing accuracy


  0%|          | 0/3129 [00:00<?, ?it/s]

100%|██████████| 3129/3129 [00:01<00:00, 2834.54it/s]

Done computing accuracy
{0: 33.33, 1: 0.0, 2: 100, 3: 33.33, 4: 66.67, 5: 100, 6: 66.67, 7: 66.67, 8: 100, 9: 0.0, 10: 100, 11: 100, 12: 100, 13: 66.67, 14: 33.33, 15: 0.0, 16: 0.0, 17: 100, 18: 66.67, 19: 100, 20: 100, 21: 100, 22: 100, 23: 100, 24: 33.33, 25: 33.33, 26: 33.33, 27: 33.33, 28: 0.0, 29: 100, 30: 33.33, 31: 100, 32: 100, 33: 100, 34: 0.0, 35: 0.0, 36: 100, 37: 66.67, 38: 100, 39: 0.0, 40: 100, 41: 100, 42: 100, 43: 100, 44: 100, 45: 66.67, 46: 100, 47: 0.0, 48: 100, 49: 100, 50: 100, 51: 0.0, 52: 100, 53: 100, 54: 100, 55: 100, 56: 0.0, 57: 33.33, 58: 100, 59: 100, 60: 100, 61: 100, 62: 0.0, 63: 100, 64: 100, 65: 100, 66: 100, 67: 0.0, 68: 0.0, 69: 0.0, 70: 0.0, 71: 33.33, 72: 0.0, 73: 100, 74: 100, 75: 100, 76: 100, 77: 100, 78: 33.33, 79: 100, 80: 0.0, 81: 100, 82: 100, 83: 33.33, 84: 100, 85: 100, 86: 100, 87: 0.0, 88: 0.0, 89: 100, 90: 100, 91: 66.67, 92: 0.0, 93: 0.0, 94: 100, 95: 100, 96: 100, 97: 100, 98: 0.0, 99: 66.67, 100: 100, 101: 100, 102: 0.0, 103: 33.33, 1

In [94]:
gt_scores=[judge.evalQA[i] for i in range(len(judge.evalQA))]

In [95]:
is_correct=[]
threshold=30
for i,x in enumerate(gt_scores):
    if x >=threshold:
        is_correct.append(1)
    else:
        is_correct.append(0)
is_correct=np.array(is_correct)

In [96]:
is_correct.shape

(3129,)

In [97]:
print(f'There are {len(is_correct)} responses. {sum(is_correct==1)} answer correctly and {sum(is_correct==0)} answer wrongly')

There are 3129 responses. 2084 answer correctly and 1045 answer wrongly


In [98]:
length=len(is_correct)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 2000
# exclude validation samples.
train_idxs = train_val_idxs[:- validset_len]  # trainset index
val_idxs = train_val_idxs[- validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_correct[test_idxs]
hall_label_wild = is_correct[train_idxs]
hall_label_val = is_correct[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:503
validset length:2000
testset length:626
trainset postive: 334 negative: 169
valset postive: 1317 negative: 683
testset postive: 433 negative: 193


In [99]:
all_embeddings = embeddings
all_embeddings.shape

(3129, 29, 3584)

In [100]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [101]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCALinear,PCAKernel,KernelPCA
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve,f1_score,balanced_accuracy_score
from ml_utils.metrics import auc_pr

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [102]:
embedding_val.shape

(2000, 29, 3584)

In [103]:
M=3584
def evaluator(n_components,i_layer,X,y):
    discriminator=KernelPCA(X[:,i_layer,:],n_components,M,method='CoP',gamma=1)
    scores=discriminator.get_score(X[:,i_layer,:])
    
    return roc_auc_score(y,scores)
    #split=discriminator.get_best_split(scores,y)
    #y_pred=scores>split
    #return accuracy_score(y,y_pred)
    #return balanced_accuracy_score(y,y_pred)
    #
    #split=discriminator.get_best_split(y)
    #y_preds=(scores>split)
    #return accuracy_score(y,y_preds)

In [104]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,4),
    'layer':range(10,embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/19 [00:00<?, ?it/s]

found best n_components:1,layer:10 Score: 0.5212454322404061
found best n_components:1,layer:11 Score: 0.5234377345024129
found best n_components:1,layer:12 Score: 0.5277734235601343
found best n_components:1,layer:13 Score: 0.5346360411379071
found best n_components:1,layer:15 Score: 0.5357733257292017
found best n_components:1,layer:17 Score: 0.5440155817994444
found best n_components:1,layer:20 Score: 0.5463679710420439
found best n_components:1,layer:21 Score: 0.5478676747699583
found best n_components:1,layer:27 Score: 0.5577663864032791


  0%|          | 0/19 [00:00<?, ?it/s]

found best n_components:2,layer:17 Score: 0.5623644402347499
found best n_components:2,layer:20 Score: 0.5624778351793364
found best n_components:2,layer:22 Score: 0.5625634372453477
found best n_components:2,layer:24 Score: 0.5665400423118784


  0%|          | 0/19 [00:00<?, ?it/s]

found best n_components:3,layer:12 Score: 0.5677051197817481
found best n_components:3,layer:13 Score: 0.5717072942965679
found best n_components:3,layer:17 Score: 0.5727484155279925


In [105]:
best_paras

{'n_components': 3, 'layer': 17, 'best_score': np.float64(0.5727484155279925)}

In [106]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [107]:
discriminator=KernelPCA(embedding_val[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(scores,hall_label_val)

In [108]:
discriminator=KernelPCA(embedding_train[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_train[:,best_layer,:])
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.3359840954274354 AUROC:0.5336073415299578


In [109]:
discriminator=KernelPCA(embedding_test[:,best_layer,:],best_paras['n_components'],M,method='CoP',gamma=1)
scores=discriminator.get_score(embedding_test[:,best_layer,:])
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.3083067092651757 AUROC:0.6140195527049503


Train Linear Prob to Get Better Results

In [110]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [111]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [112]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} f1:{f1_score(y_test,pred_test)} auroc:{roc_auc_score(y_test,scores_test)}  b-acc:{balanced_accuracy_score(y_test,pred_test)} auc-pr:{auc_pr(y_test,scores_test)}')

test set acc: 0.3083067092651757 f1:0.0 auroc:0.4410606803958406  b-acc:0.5 auc-pr:0.6131639213444567
